In [1]:
# import required libraries

import os
import supabase
from nomic import atlas
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
import numpy as np
import time
import pandas as pd

2023-09-07 11:57:29,274:INFO - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-09-07 11:57:29,274:INFO - NumExpr defaulting to 8 threads.


In [2]:
# loading environment variables

env_path = "../.env"
load_dotenv(dotenv_path=env_path)

True

In [3]:
# initialize supabase client

url = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_API_KEY")

supabase_client = supabase.create_client(url, key)

In [22]:
# query data for one course for testing
course = 'cropwizard'
response = supabase_client.table("llm-convo-monitor").select("*").eq("course_name", course).execute()
data = response.data
df = pd.DataFrame(data)
df

,id,created_at,convo,convo_id,course_name,user_email
0,5200,2023-09-07T17:03:47.705812+00:00,"{'id': 'abd2e044-fbff-455e-8c60-755cc7635182',...",abd2e044-fbff-455e-8c60-755cc7635182,cropwizard,avd6@illinois.edu
1,5201,2023-09-07T17:05:25.863567+00:00,"{'id': '3e5d4861-b128-4c64-96ac-87c74f3217e5',...",3e5d4861-b128-4c64-96ac-87c74f3217e5,cropwizard,avd6@illinois.edu
2,5216,2023-09-07T17:18:32.197379+00:00,"{'id': '43ee631a-cb58-43f5-b2af-a5b91b7585cd',...",43ee631a-cb58-43f5-b2af-a5b91b7585cd,cropwizard,avd6@illinois.edu
3,5212,2023-09-07T17:16:34.267931+00:00,"{'id': '0129ea46-207f-47e3-be90-da143857000f',...",0129ea46-207f-47e3-be90-da143857000f,cropwizard,avd6@illinois.edu
4,5217,2023-09-07T17:19:00.681823+00:00,"{'id': 'c6b4e4d8-4de7-4387-b4e9-411084dffea6',...",c6b4e4d8-4de7-4387-b4e9-411084dffea6,cropwizard,avd6@illinois.edu
5,5223,2023-09-07T17:22:38.970643+00:00,"{'id': 'b5500763-7e7b-4b23-9031-cc320a51ccbf',...",b5500763-7e7b-4b23-9031-cc320a51ccbf,cropwizard,avd6@illinois.edu
6,5227,2023-09-07T17:24:10.362647+00:00,"{'id': 'd410955f-4398-4869-b395-e6b659cc2d06',...",d410955f-4398-4869-b395-e6b659cc2d06,cropwizard,avd6@illinois.edu
7,5209,2023-09-07T17:14:43.518617+00:00,"{'id': '0ecd2c05-772a-42aa-b29a-0a892bd0e9ab',...",0ecd2c05-772a-42aa-b29a-0a892bd0e9ab,cropwizard,avd6@illinois.edu
8,5222,2023-09-07T17:21:29.223343+00:00,"{'id': 'c82056a0-2d67-4ce8-82e3-86a30f1f6dc0',...",c82056a0-2d67-4ce8-82e3-86a30f1f6dc0,cropwizard,avd6@illinois.edu
9,5224,2023-09-07T17:22:54.856839+00:00,"{'id': '2316bbd7-61f3-44aa-a79e-bb42bd688c47',...",2316bbd7-61f3-44aa-a79e-bb42bd688c47,cropwizard,avd6@illinois.edu


In [23]:
course_df = df[df['course_name'] == course]['convo']
course_df.head()

0    {'id': 'abd2e044-fbff-455e-8c60-755cc7635182',...
1    {'id': '3e5d4861-b128-4c64-96ac-87c74f3217e5',...
2    {'id': '43ee631a-cb58-43f5-b2af-a5b91b7585cd',...
3    {'id': '0129ea46-207f-47e3-be90-da143857000f',...
4    {'id': 'c6b4e4d8-4de7-4387-b4e9-411084dffea6',...
Name: convo, dtype: object

In [19]:
print(course_df[0]['messages'][1])

{'role': 'assistant', 'content': "The U.S. Environmental Protection Agency (EPA)'s mission is to protect human health and the environment. As part of this mission, the EPA is involved in efforts such as developing strategies to protect endangered and threatened species from potential impacts of agricultural practices, including the use of herbicides. For instance, the EPA has released a draft Herbicide Strategy for public comment, aimed at proposing early mitigations for more than 900 listed species and designated critical habitats to reduce potential impacts from the agricultural use of herbicides(1^,2^,3^,4^).\n\n1. University of Illinois Extension\n2. EPA releases draft herbicide strategy\n3. EPA releases draft herbicide strategy\n4. extension.pdf, page: 3", 'contexts': [{'base_url': 'https://extension.illinois.edu/newsletters/illinois-pesticide-review-newsletter/julyaugust-2023', 'course_name ': 'cropwizard', 'pagenumber': '', 'readable_filename': 'University of Illinois Extension'

In [32]:
# user email is in DF, outside of convo

## Mapping Conversations to Nomic

In [31]:
user_queries = []
metadata = []
i = 1

# log conversation instead of individual messages
for index, row in df.iterrows():
    user_email = row['user_email']
    convo = row['convo']
    messages = convo['messages']
    first_message = messages[0]['content']
    user_queries.append(first_message)
    # create metadata for multi-turn conversation
    conversation = ""
    for message in messages:
        # string of role: content, role: content, ...
        conversation += "\n>>> " + message['role'] + ": " + message['content'] + "\n"
    # add to metadata
    metadata_row = {"course": row['course_name'], "conversation": conversation, "conversation_id": convo['id'], 
                    "id": i, "user_email": user_email, "first_query": first_message}
    metadata.append(metadata_row)
    i += 1

print(len(user_queries))
print(len(metadata))

22
22


In [32]:
print(metadata[1])

{'course': 'cropwizard', 'conversation': '\n>>> user: What type of pest is most harmful to corn crops in Illinois and how can I prevent it?\n\n>>> assistant: The most harmful pest to corn crops in Illinois is the corn rootworm. Corn rootworms, specifically the western corn rootworm (Diabrotica virgifera virgifera), are the most impactful species in Illinois and the North Central U.S. These pests can cause significant damage to corn plants by feeding on the roots, leading to reduced plant vigor and yield.\n\nTo prevent corn rootworm infestations, several measures can be taken:\n\n1. Crop rotation: Implementing a crop rotation strategy by alternating corn with non-host crops, such as soybeans, can help reduce corn rootworm populations. The larvae are not capable of surviving on soybean or other broadleaf plants, so rotating crops disrupts their life cycle.\n\n2. Utilize Bt hybrids: Planting corn hybrids that have been genetically modified to express Bacillus thuringiensis (Bt) proteins t

In [33]:
# initialize langchain OpenAI embeddings model
embeddings_model = OpenAIEmbeddings()
metadata = pd.DataFrame(metadata)
embeddings = embeddings_model.embed_documents(user_queries)
embeddings = np.array(embeddings)
print(embeddings.shape)

(22, 1536)


In [34]:
# create an Atlas project
project_name = "Conversation Map for " + course
index_name = course + "_convo_index_2"
project = atlas.map_embeddings(embeddings=np.array(embeddings),
                                data=metadata,
                                id_field='id',
                                build_topic_model=True,
                                topic_label_field='first_query',
                                name=project_name,
                                colorable_fields=['conversation_id', 'first_query'])
print(project.maps)

project.create_index(index_name, build_topic_model=True)

2023-09-07 14:53:44.900 | INFO     | nomic.project:_create_project:779 - Creating project `Conversation Map for cropwizard` in organization `dabholkar.asmita`
2023-09-07 14:53:45.794 | INFO     | nomic.atlas:map_embeddings:107 - Uploading embeddings to Atlas.
2023-09-07 14:53:45.798 | WARNING  | nomic.project:_validate_and_correct_arrow_upload:261 - id_field is not a string. Converting to string from int32
1it [00:00,  1.07it/s]
2023-09-07 14:53:46.743 | INFO     | nomic.project:_add_data:1401 - Upload succeeded.
2023-09-07 14:53:46.744 | INFO     | nomic.atlas:map_embeddings:126 - Embedding upload succeeded.
2023-09-07 14:53:47.568 | INFO     | nomic.project:create_index:1111 - Created map `Conversation Map for cropwizard` in project `Conversation Map for cropwizard`: https://atlas.nomic.ai/map/27ba0013-71ba-4fd6-a710-187d114442ab/f74b45b3-7213-45df-a4cb-c16ec9feb8b7
2023-09-07 14:53:47.570 | INFO     | nomic.atlas:map_embeddings:139 - Conversation Map for cropwizard: https://atlas.no

[Conversation Map for cropwizard: https://atlas.nomic.ai/map/27ba0013-71ba-4fd6-a710-187d114442ab/f74b45b3-7213-45df-a4cb-c16ec9feb8b7]


2023-09-07 14:53:48.872 | INFO     | nomic.project:create_index:1111 - Created map `cropwizard_convo_index_2` in project `Conversation Map for cropwizard`: https://atlas.nomic.ai/map/27ba0013-71ba-4fd6-a710-187d114442ab/3336aa7f-5995-4f02-831b-7161fd0c0b71


cropwizard_convo_index_2: https://atlas.nomic.ai/map/27ba0013-71ba-4fd6-a710-187d114442ab/3336aa7f-5995-4f02-831b-7161fd0c0b71

## Mapping Query-Response Pairs to Nomic

In [8]:
user_queries = []
metadata = []
i = 1
for convo in course_df:
    messages = convo['messages']
    print(len(messages))
    pair_check = 0
    for message in messages:
        if message['role'] == 'user' and pair_check == 0:
            query = message['content']
            metadata_row = {'course': course, 'query': message['content']}
            #print("metadata row: ", metadata_row)
            pair_check += 1
        if message['role'] == 'assistant' and pair_check == 1:
            metadata_row['response'] = message['content']
            metadata_row['id'] = i 
            #print("response metadata row: ", metadata_row)
            i += 1
            pair_check += 1
        if pair_check == 2:
            # one conversation pair is complete
            user_queries.append(query)
            metadata.append(metadata_row)
            pair_check = 0

        
print("user queries: ", len(user_queries))
print("metadata", len(metadata))
print("------------------------")
    

2
user queries:  1
metadata 1
------------------------


In [12]:
# initialize langchain OpenAI embeddings model
embeddings_model = OpenAIEmbeddings()
metadata = pd.DataFrame(metadata)
embeddings = embeddings_model.embed_documents(user_queries)
embeddings = np.array(embeddings)
print(embeddings.shape)

(590, 1536)


In [13]:
# create an Atlas project
project_name = "Query-Response Map for " + course
index_name = course + "_qr_index"
project = atlas.map_embeddings(embeddings=np.array(embeddings),
                                data=metadata,
                                id_field='id',
                                build_topic_model=True,
                                topic_label_field='query',
                                name=project_name,
                                colorable_fields=['query'])
print(project.maps)

project.create_index(index_name, build_topic_model=True)


2023-08-31 19:55:40.276 | INFO     | nomic.project:_create_project:779 - Creating project `Query-Response Map for ECE408FA23` in organization `dabholkar.asmita`
2023-08-31 19:55:41.466 | INFO     | nomic.atlas:map_embeddings:107 - Uploading embeddings to Atlas.
2023-08-31 19:55:41.491 | WARNING  | nomic.project:_validate_and_correct_arrow_upload:261 - id_field is not a string. Converting to string from int32
1it [00:02,  2.43s/it]
2023-08-31 19:55:43.932 | INFO     | nomic.project:_add_data:1401 - Upload succeeded.
2023-08-31 19:55:43.932 | INFO     | nomic.atlas:map_embeddings:126 - Embedding upload succeeded.
2023-08-31 19:55:45.475 | INFO     | nomic.project:create_index:1111 - Created map `Query-Response Map for ECE408FA23` in project `Query-Response Map for ECE408FA23`: https://atlas.nomic.ai/map/d8e02f68-501e-4a25-9060-502710c082ce/b55cdca3-3ff4-454e-8f1b-a73ebc22cdff
2023-08-31 19:55:45.480 | INFO     | nomic.atlas:map_embeddings:139 - Query-Response Map for ECE408FA23: https://

[Query-Response Map for ECE408FA23: https://atlas.nomic.ai/map/d8e02f68-501e-4a25-9060-502710c082ce/b55cdca3-3ff4-454e-8f1b-a73ebc22cdff]


2023-08-31 19:55:47.813 | INFO     | nomic.project:create_index:1111 - Created map `ECE408FA23_qr_index` in project `Query-Response Map for ECE408FA23`: https://atlas.nomic.ai/map/d8e02f68-501e-4a25-9060-502710c082ce/ff7276fc-942f-45cd-a199-e19a6e941db1


ECE408FA23_qr_index: https://atlas.nomic.ai/map/d8e02f68-501e-4a25-9060-502710c082ce/ff7276fc-942f-45cd-a199-e19a6e941db1

In [51]:
# cell for all course map creation

response = supabase_client.table("llm-convo-monitor").select("*").execute()
data = response.data
print(len(data))

1571


In [53]:
df = pd.DataFrame(data)
course_names = df['course_name'].unique()
len(course_names)

126

In [54]:
low_volume_courses = []
high_volume_courses = []
for course in course_names:
    if course is None or course == 'ece408':
        continue
    
    user_queries = []
    metadata = []
    i = 1
    course_df = df[df['course_name'] == course]['convo']
    for convo in course_df: # iterate through all conversations in a course
        messages = convo['messages']

        # form query-response pairs out of the messages
        pair_check = 0
        for message in messages:
            if message['role'] == 'user' and pair_check == 0:
                query = message['content']
                metadata_row = {'course': course, 'query': message['content']}
                #print("metadata row: ", metadata_row)
                pair_check += 1
            
            if message['role'] == 'assistant' and pair_check == 1:
                metadata_row['response'] = message['content']
                metadata_row['id'] = i 
                #print("response metadata row: ", metadata_row)
                i += 1
                pair_check += 1

            if pair_check == 2:
                # one conversation pair is complete
                user_queries.append(query)
                metadata.append(metadata_row)
                pair_check = 0
    # after iterating every conversation in a course, create the map
    if len(user_queries) < 20:
        low_volume_courses.append(course)
        continue

    if len(user_queries) > 500:
        high_volume_courses.append(course)
        continue
        
    metadata = pd.DataFrame(metadata)
    embeddings = embeddings_model.embed_documents(user_queries)
    embeddings = np.array(embeddings)
    print("course name: ", course)
    print(embeddings.shape)

    # create an Atlas project
    project_name = "Query-Response Map for " + course
    index_name = course + "_qr_index"
    project = atlas.map_embeddings(embeddings=np.array(embeddings),
                                data=metadata,
                                id_field='id',
                                build_topic_model=True,
                                topic_label_field='query',
                                name=project_name,
                                colorable_fields=['query'])
    print(project.maps)

    project.create_index(index_name, build_topic_model=True)

            


course name:  badm_550_ashley
(51, 1536)


2023-08-30 15:26:29.701 | INFO     | nomic.project:_create_project:779 - Creating project `Query-Response Map for badm_550_ashley` in organization `dabholkar.asmita`
2023-08-30 15:26:31.242 | INFO     | nomic.atlas:map_embeddings:107 - Uploading embeddings to Atlas.
2023-08-30 15:26:31.255 | WARNING  | nomic.project:_validate_and_correct_arrow_upload:261 - id_field is not a string. Converting to string from int32
1it [00:00,  1.03it/s]
2023-08-30 15:26:32.239 | INFO     | nomic.project:_add_data:1401 - Upload succeeded.
2023-08-30 15:26:32.241 | INFO     | nomic.atlas:map_embeddings:126 - Embedding upload succeeded.
2023-08-30 15:26:33.498 | INFO     | nomic.project:create_index:1111 - Created map `Query-Response Map for badm_550_ashley` in project `Query-Response Map for badm_550_ashley`: https://atlas.nomic.ai/map/e7fe1faf-be7a-4ca7-b89b-22863fffe432/79d4480a-c4e5-4920-95cd-ed831d45f5d0
2023-08-30 15:26:33.500 | INFO     | nomic.atlas:map_embeddings:139 - Query-Response Map for badm_

[Query-Response Map for badm_550_ashley: https://atlas.nomic.ai/map/e7fe1faf-be7a-4ca7-b89b-22863fffe432/79d4480a-c4e5-4920-95cd-ed831d45f5d0]


2023-08-30 15:26:35.272 | INFO     | nomic.project:create_index:1111 - Created map `badm_550_ashley_qr_index` in project `Query-Response Map for badm_550_ashley`: https://atlas.nomic.ai/map/e7fe1faf-be7a-4ca7-b89b-22863fffe432/2b997f7c-0084-4db7-8e9a-76eeb62d715b


course name:  ece120
(298, 1536)


...

2023-08-30 15:30:33.104 | INFO     | nomic.project:create_index:1111 - Created map `NCSADelta_qr_index` in project `Query-Response Map for NCSADelta`: https://atlas.nomic.ai/map/f0c381d5-e2e9-41fb-807a-ae61bc818122/b6e64fef-a829-435f-89b5-ed1a44c05202
2023-08-30 15:30:33,214:INFO - error_code=rate_limit_exceeded error_message='Rate limit reached for default-text-embedding-ada-002 in organization org-4dafnkfdjctuHfuAPPkPsN46 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.' error_param=None error_type=requests message='OpenAI API error received' stream_error=False
2023-08-30 15:30:33,214:WARNING - Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for d

course name:  NuclGPT-v1
(25, 1536)


2023-08-30 15:30:49.297 | INFO     | nomic.project:_create_project:779 - Creating project `Query-Response Map for NuclGPT-v1` in organization `dabholkar.asmita`
2023-08-30 15:30:50.216 | INFO     | nomic.atlas:map_embeddings:107 - Uploading embeddings to Atlas.
2023-08-30 15:30:50.222 | WARNING  | nomic.project:_validate_and_correct_arrow_upload:261 - id_field is not a string. Converting to string from int32
1it [00:00,  1.23it/s]
2023-08-30 15:30:51.043 | INFO     | nomic.project:_add_data:1401 - Upload succeeded.
2023-08-30 15:30:51.043 | INFO     | nomic.atlas:map_embeddings:126 - Embedding upload succeeded.
2023-08-30 15:30:52.360 | INFO     | nomic.project:create_index:1111 - Created map `Query-Response Map for NuclGPT-v1` in project `Query-Response Map for NuclGPT-v1`: https://atlas.nomic.ai/map/76043322-073c-4d8c-8c90-8e563da8564f/335097b0-a80d-4f38-8559-c6075a46a8b2
2023-08-30 15:30:52.360 | INFO     | nomic.atlas:map_embeddings:139 - Query-Response Map for NuclGPT-v1: https://

[Query-Response Map for NuclGPT-v1: https://atlas.nomic.ai/map/76043322-073c-4d8c-8c90-8e563da8564f/335097b0-a80d-4f38-8559-c6075a46a8b2]


2023-08-30 15:30:53.900 | INFO     | nomic.project:create_index:1111 - Created map `NuclGPT-v1_qr_index` in project `Query-Response Map for NuclGPT-v1`: https://atlas.nomic.ai/map/76043322-073c-4d8c-8c90-8e563da8564f/441b7ff6-00ef-47f3-98a9-e45d327a1414


In [55]:
print("low volume courses: ", low_volume_courses)
print("high volume courses: ", high_volume_courses)

low volume courses:  ['test-video-ingest', 'badm-567', 'test-video-ingest-28', 'pdeploy999', 'badm-350-summer', 'previewtesting1', 'localtest2', 'your-favorite-url', 'mantine', 'test-video-ingest-2', 'badm_567_v2', 'erpnext', 'mip', 'farmdoc_test_kastan-v1', 'personalstatement', 'hrc', 'csv', 'star_nox', 'badm_567', 'SPaRCEd', 'localdemo8', 'badm_567_thumbnails', 'chatbot', 'erp', 'extreme', 'rohan_atree', 'zotero-extreme', 'test-video-ingest-20', 'gies-online-mba2', 'gies-online-mba', 'careerassistant', 'weather', 'lillian-wang-blog', 'local-test5', 'demo-for-vyriad', 'ag-gpt-beta', 'rtest', 'previewdeploy', 'r2test', 'personal-statement', 'rohan_excel', 'langchain-python', 'langchain', 'ncsa-live-demo', 'rohan_atree_individual', 'meta11-test', 'HealthyLivingGuide', 'rohan', 'babreu', 'test-video-ingest-31', 'p', 'test-video-ingest-17', 'summary', 'test-video-ingest-3', 'test-video-ingest-27', 'lillian-wang-blog-2', 'python-magic', 'ansible2', 'ece408fa23', 'farmdoc_test_josh_v2', 'lo